In [1]:
from pokemon import *
from pokemon_statistics import *
from api_helper import runSim
import sqlite3
import requests
import pickle

In [2]:
def makeFormatString(poke_name, moves=[], maxmoves=[], ability='', item=''):
    # I should check the poke_name against table names in database or 
    # else ill get sql injected problem here, granted the consequences are small 
    # but lets be honest it also probably isnt
    # put in a validate statement like if(validate(poke_name))
        
    format_string = 'SELECT * FROM ' + poke_name 
    format_tuple = ()
    
    where = False
    
    for move in moves:
        if where:
            format_string += ' AND (move1=? OR move2=? OR move3=? OR move4=?)'
        else:
            format_string += ' WHERE (move1=? OR move2=? OR move3=? OR move4=?)'
            where = True
        format_tuple += (move, move, move, move)
    
    for move in maxmoves:
        if where:
            format_string += ' AND (maxmove1=? OR maxmove2=? OR maxmove3=? OR maxmove4=?)'
        else:
            format_string += ' WHERE (maxmove1=? OR maxmove2=? OR maxmove3=? OR maxmove4=?)'
            where = True
        format_tuple += (move, move, move, move)
                
    if ability_name != '':
        if where: 
            format_string += ' AND ability=?'
        else:
            format_string += ' WHERE ability=?'
            where = True
        format_tuple += (ability_name,)
        
    if item_name != '':
        if where: 
            format_string += ' AND item=?'
        else:
            format_string += ' WHERE item=?'
            where = True
        format_tuple += (item_name,)
    
    
    return format_string, format_tuple

def update_freq(freq, key):
    if key in freq:
        freq[key] += 1
    else:
        freq[key] = 1
        
def update_predictions(pokemon):
    format_string, format_tuple = makeFormatString(pokemon.name, pokemon.moves, pokemon.max_moves, 
                                                   pokemon.ability, pokemon.item)
    cursor.execute(format_string, format_tuple)
    
    total = 0
    move_freq = {}
    ability_freq = {}
    item_freq = {}
    for (num_id, ability, item, move1, move2, move3, move4, _, _, _, _) in cursor:
        total += 1

        update_freq(ability_freq, ability)
        update_freq(item_freq, item)

        update_freq(move_freq, move1)
        update_freq(move_freq, move2)
        update_freq(move_freq, move3)
        update_freq(move_freq, move4)
    
    if "UNKNOWN" in ability_freq.keys():
        ability_freq.pop("UNKNOWN")
    if "UNKNOWN" in item_freq.keys():
        item_freq.pop("UNKNOWN")
    if "UNKNOWN" in move_freq.keys():
        move_freq.pop("UNKNOWN")
    if "UNKNOWN" in maxmove_freq.keys():
        maxmove_freq.pop("UNKNOWN")
    
    if (pokemon.ability == ''):
        key, _ = max(ability_freq.items(), key=lambda x:x[1])
        pokemon.pred_ability = key
    
    if (pokemon.item == ''):
        key, _ = max(item_freq.items(), key=lambda x:x[1])
        pokemon.pred_item = key
    
    pokemon.pred_moves = []
    for i in range(4 - len(pokemon.moves)):
        key, _ = max(move_freq.items(), key=lambda x:x[1])
        pokemon.pred_moves.append(key)
        move_freq.pop(key)
    

In [3]:
connection = sqlite3.connect("pokemon_profiles.db")
cursor = connection.cursor()

# Testing below this point

In [4]:
ms = state()
ms.team1 = team()
ms.team2 = team()

In [5]:
ms.team1.side = 'p1'
ms.team1.full = []
ms.team1.full.append(createPikalyticsPokemon('incineroar'))
ms.team1.full.append(createPikalyticsPokemon('regieleki'))
ms.team1.full.append(createPikalyticsPokemon('landorus-therian'))
ms.team1.full.append(createPikalyticsPokemon('rillaboom'))
ms.team1.full.append(createPikalyticsPokemon('urshifu-rapid-strike'))
ms.team1.full.append(createPikalyticsPokemon('porygon2'))

ms.team1.active.append(ms.team1.full[0])
ms.team1.active.append(ms.team1.full[1])

In [6]:
ms.team2.side = 'p2'
ms.team2.full = []
ms.team2.full.append(createPikalyticsPokemon('urshifu'))
ms.team2.full.append(createPikalyticsPokemon('grimmsnarl'))
ms.team2.full.append(createPikalyticsPokemon('torkoal'))
ms.team2.full.append(createPikalyticsPokemon('venusaur'))
ms.team2.full.append(createPikalyticsPokemon('thundurus'))
ms.team2.full.append(createPikalyticsPokemon('tapu fini'))

ms.team2.active.append(ms.team2.full[0])
ms.team2.active.append(ms.team2.full[1])

In [18]:
%%time
runSim(ms, [['>p1 move 2 1, move 4\n', '>p2 move 1 1, move 2\n']], num_sims=100)

12.19774011299436 [['>p1 move 2 1, move 4\n', '>p2 move 1 1, move 2\n']]
CPU times: user 98.2 ms, sys: 757 ms, total: 855 ms
Wall time: 8.09 s


In [8]:
import pickle
f = open("movedex.pkl", "rb")
movedex = pickle.load(f)
f.close()

In [32]:
def genMoveCombos(pokes, side):
    poke1 = pokes[0]
    poke2 = pokes[1]
    
    base = ">" + side
    combos = []
    
    p1 = poke1.moves + poke1.pred_moves[:4 - len(poke1.moves)]
    p2 = poke2.moves + poke2.pred_moves[:4 - len(poke2.moves)]
    
    moves1 = []
    moves2 = []
    
    for m1 in p1:
        moves1.append(targetChoices(m1, ' -2'))
        
    for m2 in p2:
        moves2.append(targetChoices(m2, ' -1'))
        
    for i1 in range(len(p1)):
        m1 = base + ' move ' + str(i1 + 1)
        for t1 in moves1[i1]:
            m2 = m1 + t1
            for i2 in range(len(p2)):
                m3 = m2 + ', move ' + str(i2 + 1)
                for t2 in moves2[i1]:
                    m4 = m3 + t2
                    combos.append(m4)

    return combos
                    
def targetChoices(move, ally):
    v = movedex[move.lower()]
    
    if v == 'adjacentAlly':
        return [ally]
    if v == 'adjacentAllyOrSelf':
        return [ally]
    if v == 'adjacentFoe':
        return [' 1', ' 2']
    if v == 'all':
        return ['']
    if v == 'allAdjacent':
        return ['']
    if v == 'allAdjacentFoes':
        return ['']
    if v == 'allies':
        return ['']
    if v == 'allySide':
        return ['']
    if v == 'allyTeam':
        return ['']
    if v == 'any':
        return [ally, ' 1', ' 2']
    if v == 'foeSide':
        return ['']
    if v == 'normal':
        return [ally, ' 1', ' 2']
    if v == 'randomNormal':
        return ['']
    if v == 'scripted':
        return ['']
    if v == 'self':
        return ['']
        

In [33]:
p1 = createPikalyticsPokemon('incineroar')
p2 = createPikalyticsPokemon('regieleki')

p3 = createPikalyticsPokemon('urshifu')
p4 = createPikalyticsPokemon('grimmsnarl')

combos = genMoveCombos([p1, p2], 'p1')
print(len(combos))

combos2 = genMoveCombos([p3, p4], 'p2')
print(len(combos2))

print(len(combos) * len(combos2))

combos

96
72
6912


['>p1 move 1 -2, move 1',
 '>p1 move 1 -2, move 2',
 '>p1 move 1 -2, move 3',
 '>p1 move 1 -2, move 4',
 '>p1 move 1 1, move 1',
 '>p1 move 1 1, move 2',
 '>p1 move 1 1, move 3',
 '>p1 move 1 1, move 4',
 '>p1 move 1 2, move 1',
 '>p1 move 1 2, move 2',
 '>p1 move 1 2, move 3',
 '>p1 move 1 2, move 4',
 '>p1 move 2 -2, move 1 -1',
 '>p1 move 2 -2, move 1 1',
 '>p1 move 2 -2, move 1 2',
 '>p1 move 2 -2, move 2 -1',
 '>p1 move 2 -2, move 2 1',
 '>p1 move 2 -2, move 2 2',
 '>p1 move 2 -2, move 3 -1',
 '>p1 move 2 -2, move 3 1',
 '>p1 move 2 -2, move 3 2',
 '>p1 move 2 -2, move 4 -1',
 '>p1 move 2 -2, move 4 1',
 '>p1 move 2 -2, move 4 2',
 '>p1 move 2 1, move 1 -1',
 '>p1 move 2 1, move 1 1',
 '>p1 move 2 1, move 1 2',
 '>p1 move 2 1, move 2 -1',
 '>p1 move 2 1, move 2 1',
 '>p1 move 2 1, move 2 2',
 '>p1 move 2 1, move 3 -1',
 '>p1 move 2 1, move 3 1',
 '>p1 move 2 1, move 3 2',
 '>p1 move 2 1, move 4 -1',
 '>p1 move 2 1, move 4 1',
 '>p1 move 2 1, move 4 2',
 '>p1 move 2 2, move 1 -1',


In [11]:
set(list(movedex.values()))

{'adjacentAlly',
 'adjacentAllyOrSelf',
 'adjacentFoe',
 'all',
 'allAdjacent',
 'allAdjacentFoes',
 'allies',
 'allySide',
 'allyTeam',
 'any',
 'foeSide',
 'normal',
 'randomNormal',
 'scripted',
 'self'}